In [ ]:
import pandas as pd 
import numpy as np 
import os 
import seaborn as sns 
import matplotlib.pyplot as plt
import math
import scipy.signal as sig

In [ ]:
def merge_bw_zv(bw_df, zv_df):

    print('total bw rows with id in video dataset') 
    print(len(zv_df))

    # merge bw data set rows with zeno videos rows 
        # merge bw data set rows with zeno videos rows 
        # id and date needs to be the same 
        # should only use each brainwalk visit once - once PWS_1 video per person 

    merged_bw_zv = []

    # Loop through each row in av_df
    for index, zv_row in zv_df.iterrows():
   
        current_id = zv_row['id_video']
        current_date = zv_row['visit_date_video']
        zv_row_df = pd.DataFrame([zv_row])

        # Find rows in brainwalk data set with same id and same date as current zv data 
        zv_in_bw_current_id_rows = bw_df[(bw_df['bw_id'] == current_id) & (bw_df['trialdate'] == current_date)]
        #zv_in_bw_current_id_date_rows = zv_in_bw_current_id_rows[zv_in_bw_current_id_rows['visit_date'] == current_date]
   
        if len(zv_in_bw_current_id_rows) == 1: 
            bw_row_to_merge = zv_in_bw_current_id_rows
            # merge bw and zv, store merged row
            merged_row = zv_row_df.merge(bw_row_to_merge, left_on='id_video', right_on='bw_id')
            merged_bw_zv.append(merged_row)
        
        # if more than one row for the id and date, pick one with least na values 
        elif len(zv_in_bw_current_id_rows) > 1:
            bw_row_to_merge = zv_in_bw_current_id_rows.loc[[zv_in_bw_current_id_rows.isna().sum(axis=1).idxmin()]]
            # merge bw and zv, store merged row
            merged_row = zv_row_df.merge(bw_row_to_merge, left_on='id_video', right_on='bw_id')
            merged_bw_zv.append(merged_row)

            print('multiple rows for the id and date combo')
            print(current_id)
            print(current_date)

        else: 
            print('No matching id and daterow from video vs mat')
            print(current_id)
            print(current_date)


    # merge all bw and zv data together 
    merged_bw_zv_df = pd.concat(merged_bw_zv)
    merged_bw_zv_df = merged_bw_zv_df.reset_index(drop=True) # reset index 

    # check same ID for each row 
    print('mismatched zeno video vs brainwalk id')
    print(sum(merged_bw_zv_df['id_video'] != merged_bw_zv_df['bw_id']))

    print('mismatched zeno video vs brainwalk date')
    print(sum(merged_bw_zv_df['visit_date_video'] != merged_bw_zv_df['trialdate']))

    # saved merged df for future reference 
#    merged_bw_zv_df.to_csv(os.path.join(out_path,  'zv_bw_merged_' + task + '.csv'))

    return merged_bw_zv_df

In [ ]:
# merge with brainwalk data - T25FW, FW velocity, PWS velocity 
bw_df = pd.read_excel(r'C:\Users\mmccu\AppData\Local\Temp\ccsecure\2025_01_24_BrainWalk_AllData_Long_MM.xlsx', 
                     index_col = None, 
                     usecols = ['bw_id', 'record_id', 'trialdate', 'visit_date', 'bingoEHR_EDSS_measure_value', 'msfcEHR_T25FW SPEED AVG',
                                'PWS_velocitycmsecmean', 'FW_velocitycmsecmean'])

In [ ]:
# convert data 

depth_proxy_describe_all['visit_date_video'] = pd.to_datetime(depth_proxy_describe_all['visit_date_video'].str.replace('_', '-'), format='%Y-%m-%d')
# add bw data to describe df 
depth_proxy_w_bw = merge_bw_zv(bw_df, depth_proxy_describe_all)

In [ ]:
def scatterplot_all_vs_one(df, column_name):
    col_1_data = df[column_name]
    for current_col_2 in df.columns:
        col_2_data =  df[current_col_2]

        plt.scatter(col_1_data, col_2_data)
        plt.xlabel(column_name)
        plt.ylabel(current_col_2) 
        plt.show()

In [ ]:
depth_proxy_w_bw_fw = depth_proxy_w_bw.loc[depth_proxy_w_bw['filename'].str.contains('FW_1')]  
depth_proxy_w_bw_pws = depth_proxy_w_bw.loc[depth_proxy_w_bw['filename'].str.contains('PWS_1')]

In [ ]:
depth_proxy_w_bw_fw.head()

In [ ]:
# plot all vars from fast walk videos vs t25fw 
scatterplot_all_vs_one(depth_proxy_w_bw_fw, 'msfcEHR_T25FW SPEED AVG')

In [ ]:
# plot all vars vs PWS velocity 
scatterplot_all_vs_one(depth_proxy_w_bw_pws, 'PWS_velocitycmsecmean')

In [ ]:
# plot all vars vs FW velocity 
scatterplot_all_vs_one(depth_proxy_w_bw_fw, 'FW_velocitycmsecmean')